# Install dan Download Category Encoder


In [ ]:
pip install category_encoders

time: 2.54 s (started: 2021-01-09 03:27:46 +00:00)


# Install Ipython Autotime

In [ ]:
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.54 s (started: 2021-01-09 03:27:48 +00:00)


# Import Libraries

In [ ]:
import keras
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import svm
from sklearn.model_selection import KFold
from category_encoders.target_encoder import TargetEncoder

from sklearn import tree
import matplotlib.pyplot as plt

from category_encoders.target_encoder import TargetEncoder

time: 11.2 ms (started: 2021-01-09 03:27:51 +00:00)


# Load Data

Read data dan drop kolom yang tidak dibutuhkan

In [ ]:
df = pd.read_csv("winemag-data_first150k.csv")
df.drop(columns=['country', 'province', 'region_1', 'variety', 'description', 'Unnamed: 0', 'region_2'], inplace=True)
df

,designation,points,price,winery
0,Martha's Vineyard,96,235.0,Heitz
1,Carodorum Selección Especial Reserva,96,110.0,Bodega Carmen Rodríguez
2,Special Selected Late Harvest,96,90.0,Macauley
3,Reserve,96,65.0,Ponzi
4,La Brûlade,95,66.0,Domaine de la Bégude
...,...,...,...,...
150925,NaN,91,20.0,Feudi di San Gregorio
150926,Cuvée Prestige,91,27.0,H.Germain
150927,Terre di Dora,91,20.0,Terredora
150928,Grand Brut Rosé,90,52.0,Gosset


time: 751 ms (started: 2021-01-09 03:27:51 +00:00)


Assign df_disc dengan data yang tidak memiliki nilai null

In [ ]:
df_disc = df.copy()
for col in df_disc.columns:
    df_disc = df_disc[df_disc[col].notnull()]

df_disc

,designation,points,price,winery
0,Martha's Vineyard,96,235.0,Heitz
1,Carodorum Selección Especial Reserva,96,110.0,Bodega Carmen Rodríguez
2,Special Selected Late Harvest,96,90.0,Macauley
3,Reserve,96,65.0,Ponzi
4,La Brûlade,95,66.0,Domaine de la Bégude
...,...,...,...,...
150923,Demi-Sec,91,30.0,Jacquart
150924,Diamant Bleu,91,70.0,Heidsieck & Co Monopole
150926,Cuvée Prestige,91,27.0,H.Germain
150927,Terre di Dora,91,20.0,Terredora


time: 85.4 ms (started: 2021-01-09 03:27:51 +00:00)


Assign df_disc dengan data yang pada kolom "winery" memiliki nilai yang duplicate

In [ ]:
df_disc = df_disc[df_disc.duplicated(subset=["designation"], keep=False)]
print(df_disc)

                 designation  points  price                   winery
0          Martha's Vineyard      96  235.0                    Heitz
3                    Reserve      96   65.0                    Ponzi
5                  Numanthia      95   73.0                Numanthia
6                  San Román      95   65.0                 Maurodos
9       Gap's Crown Vineyard      95   60.0                Blue Farm
...                      ...     ...    ...                      ...
150923              Demi-Sec      91   30.0                 Jacquart
150924          Diamant Bleu      91   70.0  Heidsieck & Co Monopole
150926        Cuvée Prestige      91   27.0                H.Germain
150927         Terre di Dora      91   20.0                Terredora
150928       Grand Brut Rosé      90   52.0                   Gosset

[83055 rows x 4 columns]
time: 47.4 ms (started: 2021-01-09 03:27:52 +00:00)


Assign df_disc dengan data yang pada kolom "designation" memiliki nilai yang duplicate

In [ ]:
df_disc = df_disc[df_disc.duplicated(subset=["winery"], keep=False)]
print(df_disc)

                 designation  points  price                   winery
0          Martha's Vineyard      96  235.0                    Heitz
3                    Reserve      96   65.0                    Ponzi
5                  Numanthia      95   73.0                Numanthia
6                  San Román      95   65.0                 Maurodos
9       Gap's Crown Vineyard      95   60.0                Blue Farm
...                      ...     ...    ...                      ...
150923              Demi-Sec      91   30.0                 Jacquart
150924          Diamant Bleu      91   70.0  Heidsieck & Co Monopole
150926        Cuvée Prestige      91   27.0                H.Germain
150927         Terre di Dora      91   20.0                Terredora
150928       Grand Brut Rosé      90   52.0                   Gosset

[81884 rows x 4 columns]
time: 35.2 ms (started: 2021-01-09 03:27:52 +00:00)


Mendiskritkan target menjadi kontinu menggunakan qcut dari pandas dengan parameter pd.qcut(kolom, jumlah_kelas, label) dan replace kolom target (price) dengan angka

In [ ]:
df_disc['points'] = pd.qcut(df_disc['points'], 2, labels=['low', 'high'])
df_disc['points'].replace({'low': 0, 'high': 1}, inplace=True)
df_disc

,designation,points,price,winery
0,Martha's Vineyard,1,235.0,Heitz
3,Reserve,1,65.0,Ponzi
5,Numanthia,1,73.0,Numanthia
6,San Román,1,65.0,Maurodos
9,Gap's Crown Vineyard,1,60.0,Blue Farm
...,...,...,...,...
150923,Demi-Sec,1,30.0,Jacquart
150924,Diamant Bleu,1,70.0,Heidsieck & Co Monopole
150926,Cuvée Prestige,1,27.0,H.Germain
150927,Terre di Dora,1,20.0,Terredora


time: 86 ms (started: 2021-01-09 03:27:52 +00:00)


# Encoding

Menggunakan Target Encoder yang meng-encode berdasarkan target. Value yang diberikan antara 0 sampai 1

In [ ]:
te = TargetEncoder()

y = df_disc['points'].values
X = df_disc.drop(columns=['points']).values
te.fit(X, y)
X = te.transform(X)

X.head()

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,0,1,2
0,0.848404,0.999991,0.848404
1,0.359072,0.781705,0.795918
2,0.999486,0.794118,1.000000
3,0.973267,0.781705,0.899943
4,0.781250,0.761444,0.973267


time: 345 ms (started: 2021-01-09 03:27:52 +00:00)


# Training

## ANN

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=1)

time: 16.3 ms (started: 2021-01-09 03:27:52 +00:00)


In [ ]:
model=keras.models.Sequential([
    keras.layers.Dense(512, input_dim = X_train.shape[1], activation='relu'),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=128, activation='relu'),

    keras.layers.Dense(units=1, activation="sigmoid"),
],name="Initial_model",)
model.summary()

Model: "Initial_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 166,401
Trainable params: 166,401
Non-trainable params: 0
_________________________________________________________________
time: 300 ms (started: 2021-01-09 03:27:52 +00:00)


In [ ]:
learning_rate = 0.005
optimizer = keras.optimizers.Adam(lr=learning_rate)

time: 1.66 ms (started: 2021-01-09 03:27:52 +00:00)


In [ ]:
model.compile(optimizer=optimizer,
            loss='binary_crossentropy', metrics=['accuracy'])

time: 34.2 ms (started: 2021-01-09 03:27:52 +00:00)


In [ ]:
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=1000,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

time: 2.77 ms (started: 2021-01-09 03:27:52 +00:00)


In [ ]:
history = model.fit(X_train, y_train,
                    epochs=500, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1, callbacks=[early_stopping_monitor])

Epoch 1/500
24/24 [==============================] - 2s 54ms/step - loss: 0.4685 - accuracy: 0.7360 - val_loss: 0.2753 - val_accuracy: 0.8769
Epoch 2/500
24/24 [==============================] - 1s 44ms/step - loss: 0.2778 - accuracy: 0.8753 - val_loss: 0.2675 - val_accuracy: 0.8800
Epoch 3/500
24/24 [==============================] - 1s 45ms/step - loss: 0.2764 - accuracy: 0.8756 - val_loss: 0.2596 - val_accuracy: 0.8812
Epoch 4/500
24/24 [==============================] - 1s 46ms/step - loss: 0.2627 - accuracy: 0.8796 - val_loss: 0.2565 - val_accuracy: 0.8797
Epoch 5/500
24/24 [==============================] - 1s 52ms/step - loss: 0.2578 - accuracy: 0.8782 - val_loss: 0.2513 - val_accuracy: 0.8816
Epoch 6/500
24/24 [==============================] - 1s 46ms/step - loss: 0.2592 - accuracy: 0.8756 - val_loss: 0.2507 - val_accuracy: 0.8819
Epoch 7/500
24/24 [==============================] - 1s 46ms/step - loss: 0.2509 - accuracy: 0.8803 - val_loss: 0.2543 - val_accuracy: 0.8779
Epoch 

In [ ]:
_, train_acc = model.evaluate(X_train, y_train, verbose=0)
_, test_acc = model.evaluate(X_test, y_test, verbose=0)

time: 3.22 s (started: 2021-01-09 03:37:31 +00:00)


In [ ]:
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Train: 0.901, Test: 0.877
time: 831 µs (started: 2021-01-09 03:37:34 +00:00)


## Decision Tree

In [ ]:
param_dt = {
    'test_size': [0.9, 0.7, 0.5, 0.3, 0.1],
    'max_depth': [3, 5, 7],
    'criterion': ['gini', 'entropy']
}

time: 2.26 ms (started: 2021-01-09 03:37:34 +00:00)


In [ ]:
row_format ="{:>10}" * (len(param_dt) + 2)
print(row_format.format('t_size', 'max_depth', 'criterion', 'akurasi', 'f1'))
acc_bfr = -math.inf
for i in param_dt['test_size']:
  for j in param_dt['max_depth']:
    for k in param_dt['criterion']:
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=42)
      dt = DecisionTreeClassifier(random_state=42, max_depth=j, criterion=k)
      dt.fit(X_train, y_train)

      # print(y_test)
      pred = dt.predict(X_test)
      acc = metrics.accuracy_score(y_test, pred)
      f1 = metrics.f1_score(y_test, pred)
      if(acc > acc_bfr):
        acc_bfr = acc
        best_param_dt = {'t_size': i, 'max_depth': j, 'criterion': k, 'accuracy': acc, 'f1': f1}
        best_dt = dt
      print(row_format.format(str(i), str(j), str(k), '%.3f' % acc, '%.3f' % f1))
print('best param: ', best_param_dt)     

    t_size max_depth criterion   akurasi        f1
       0.9         3      gini     0.863     0.855
       0.9         3   entropy     0.857     0.851
       0.9         5      gini     0.879     0.862
       0.9         5   entropy     0.877     0.860
       0.9         7      gini     0.877     0.860
       0.9         7   entropy     0.877     0.859
       0.7         3      gini     0.865     0.857
       0.7         3   entropy     0.855     0.818
       0.7         5      gini     0.879     0.860
       0.7         5   entropy     0.872     0.844
       0.7         7      gini     0.880     0.864
       0.7         7   entropy     0.876     0.851
       0.5         3      gini     0.878     0.864
       0.5         3   entropy     0.859     0.825
       0.5         5      gini     0.882     0.868
       0.5         5   entropy     0.874     0.848
       0.5         7      gini     0.881     0.862
       0.5         7   entropy     0.883     0.868
       0.3         3      gini 

## SVM

In [ ]:
param_svm = {
  'test_size': [0.9, 0.7, 0.5, 0.3, 0.1],  
  'C'    : [1, 10, 100, 0.1],
  'gamma': [0.1, 0.01, 0.001, 1.0],
  'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
}

time: 3.49 ms (started: 2021-01-09 03:37:36 +00:00)


In [ ]:
row_format ="{:>10}" * (len(param_svm) + 2)
print(row_format.format(*param_svm, 'akurasi', 'f1'))
acc_bfr = -math.inf
for i in param_svm['test_size']:
  for j in param_svm['C']:
    for k in param_svm['gamma']:
      for l in param_svm['kernel']:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=42)
        svc = SVC(C=j, gamma=k, kernel=l)
        svc.fit(X_train, y_train)

        # print(y_test)
        pred = svc.predict(X_test)
        acc = metrics.accuracy_score(y_test, pred)
        f1 = metrics.f1_score(y_test, pred)
        if(acc > acc_bfr):
          acc_bfr = acc
          best_param_svm = {'t_size': i, 'C': j, 'gamma': k, 'kernel': l, 'accuracy': acc, 'f1': f1}
        print(row_format.format(str(i), str(j), str(k), str(l), '%.3f' % acc, '%.3f' % f1))
print('best param: ', best_param_svm)     

 test_size         C     gamma    kernel   akurasi        f1
       0.9         1       0.1    linear     0.879     0.860
       0.9         1       0.1      poly     0.856     0.814
       0.9         1       0.1       rbf     0.879     0.860
       0.9         1       0.1   sigmoid     0.879     0.860
       0.9         1      0.01    linear     0.879     0.860
       0.9         1      0.01      poly     0.564     0.000
       0.9         1      0.01       rbf     0.879     0.859
       0.9         1      0.01   sigmoid     0.879     0.859
       0.9         1     0.001    linear     0.879     0.860
       0.9         1     0.001      poly     0.564     0.000
       0.9         1     0.001       rbf     0.878     0.856
       0.9         1     0.001   sigmoid     0.877     0.853
       0.9         1       1.0    linear     0.879     0.860
       0.9         1       1.0      poly     0.877     0.852
       0.9         1       1.0       rbf     0.881     0.862
       0.9         1    

## KNN

In [ ]:
param_knn = {
  'test_size': [0.9, 0.7, 0.5, 0.3, 0.1],  
  'n_neighbor': [3, 5],
  'weights': ['uniform', 'distance'],
  'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
}

time: 1.59 ms (started: 2021-01-09 07:18:19 +00:00)


In [ ]:
row_format ="{:>10}" * (len(param_knn) + 2)
print(row_format.format(*param_knn, 'akurasi', 'f1'))
acc_bfr = -math.inf
for i in param_knn['test_size']:
  for j in param_knn['n_neighbor']:
    for k in param_knn['weights']:
      for l in param_knn['algorithm']:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=i, random_state=42)
        knn = KNeighborsClassifier(n_neighbors=j, weights=k, algorithm=l)
        knn.fit(X_train, y_train)

        # print(y_test)
        pred = knn.predict(X_test)
        acc = metrics.accuracy_score(y_test, pred)
        f1 = metrics.f1_score(y_test, pred)
        if(acc > acc_bfr):
          acc_bfr = acc
          best_param_knn = {'t_size': i, 'n_beighbors': j, 'weights': k, 'algorithm': l, 'accuracy': acc, 'f1': f1}
        print(row_format.format(str(i), str(j), str(k), str(l), '%.3f' % acc, '%.3f' % f1))
print('best param: ', best_param_knn)     

 test_sizen_neighbor   weights algorithm   akurasi        f1
       0.9         3   uniform      auto     0.865     0.846
       0.9         3   uniform ball_tree     0.865     0.845
       0.9         3   uniform   kd_tree     0.865     0.846
       0.9         3   uniform     brute     0.865     0.846
       0.9         3  distance      auto     0.865     0.846
       0.9         3  distance ball_tree     0.865     0.846
       0.9         3  distance   kd_tree     0.865     0.846
       0.9         3  distance     brute     0.865     0.846
       0.9         5   uniform      auto     0.871     0.852
       0.9         5   uniform ball_tree     0.871     0.851
       0.9         5   uniform   kd_tree     0.871     0.852
       0.9         5   uniform     brute     0.871     0.851
       0.9         5  distance      auto     0.870     0.851
       0.9         5  distance ball_tree     0.870     0.851
       0.9         5  distance   kd_tree     0.870     0.851
       0.9         5  di